In [1]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Input, Dense, Flatten, Activation, Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Reshape
from tensorflow.keras.models import Model, Sequential
import numpy as np
import pandas as pd
import tensorflow as tf
import logging
import os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
def is_nan(x):
    return x is np.nan or x != x


In [4]:
data = pd.read_csv('../data/more_label-FullLabel.csv')
html_name = data['html']

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4927 entries, 0 to 4926
Data columns (total 18 columns):
Unnamed: 0              4927 non-null int64
Unnamed: 0.1            4927 non-null int64
url                     4927 non-null object
categories              799 non-null object
is entity               1722 non-null float64
html                    4927 non-null object
render                  4927 non-null object
media_introduction      4927 non-null float64
others                  4927 non-null float64
location                4927 non-null float64
social_media_profile    4927 non-null float64
encyclopedia            4927 non-null float64
qa_forum                4927 non-null float64
shopping_item           4927 non-null float64
list                    4927 non-null float64
media_player            4927 non-null float64
article                 4927 non-null float64
NewCategory             4927 non-null object
dtypes: float64(11), int64(2), object(5)
memory usage: 693.0+ KB


In [5]:
maxlen = 300 # sequence 取到的长度，用于下面的padding
input_dir = '/home/yujie6cs/vectors/'
count = 0
x = []

import re
pattern = re.compile(r'\d+')

for id in np.arange(len(html_name)):
    count += 1
    file_name = 'page' + pattern.findall(html_name[id])[0] + '.npy'
    this_x = np.load(os.path.join(input_dir, file_name))
    this_x = this_x.tolist()
    x.append(this_x)
    if count % 1000 == 0:
        logging.info('{} has loaded...'.format(count))
print('load {} vectors done'.format(count))
x = sequence.pad_sequences(x, maxlen=maxlen)

2020-01-16 03:30:15,755 : INFO : 1000 has loaded...
2020-01-16 03:30:16,119 : INFO : 2000 has loaded...
2020-01-16 03:30:16,456 : INFO : 3000 has loaded...
2020-01-16 03:30:16,816 : INFO : 4000 has loaded...


load 4927 vectors done


In [6]:
cate = ['media_introduction', 'others', 'location', 'social_media_profile', 'encyclopedia', \
            'qa_forum', 'shopping_item', 'list', 'media_player', 'article']
y_train = data[cate]

print(x.shape, y_train.shape)

(4927, 300) (4927, 10)


In [21]:
from tensorflow.keras import regularizers

def GetModel(input_shape=(300,)):
    input_vec = Input(shape=input_shape)
    layer1 = Dense(500, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_vec)
    layer2 = Dense(250, activation='relu')(layer1)
    layer3 = Dense(125, activation='relu')(layer2)
    layer4 = Dense(50, activation='relu')(layer3)
    output = Dense(10, activation='softmax')(layer4)
    
    clf = Model(input_vec, output)
    clf.compile(optimizer=tf.keras.optimizers.Adam(0.00001),
              loss='categorical_crossentropy',
              metrics=['acc']) # may add f1_score later
    clf.summary()
    return clf

html_clf = GetModel()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
dense_13 (Dense)             (None, 500)               150500    
_________________________________________________________________
dense_14 (Dense)             (None, 250)               125250    
_________________________________________________________________
dense_15 (Dense)             (None, 125)               31375     
_________________________________________________________________
dense_16 (Dense)             (None, 50)                6300      
_________________________________________________________________
dense_17 (Dense)             (None, 10)                510       
Total params: 313,935
Trainable params: 313,935
Non-trainable params: 0
_____________________________________________________

In [23]:
html_clf.fit(x, y_train, batch_size=512, epochs=400, verbose=1,
                        shuffle=True, validation_split=0.20)

html_clf.save('../saved_model/html_clf.h5')

Train on 3941 samples, validate on 986 samples
Epoch 1/400
3941/3941 [==============================] - 0s 15us/sample - loss: 1.2227 - acc: 0.8059 - val_loss: 0.9575 - val_acc: 0.9361
Epoch 2/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.2220 - acc: 0.8061 - val_loss: 0.9578 - val_acc: 0.9361
Epoch 3/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.2215 - acc: 0.8061 - val_loss: 0.9579 - val_acc: 0.9361
Epoch 4/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.2209 - acc: 0.8064 - val_loss: 0.9578 - val_acc: 0.9361
Epoch 5/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.2204 - acc: 0.8064 - val_loss: 0.9582 - val_acc: 0.9361
Epoch 6/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.2198 - acc: 0.8064 - val_loss: 0.9579 - val_acc: 0.9351
Epoch 7/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.2192 - acc: 0.8064 - val_loss: 0.9579 - val

3941/3941 [==============================] - 0s 14us/sample - loss: 1.1913 - acc: 0.8115 - val_loss: 0.9583 - val_acc: 0.9351
Epoch 60/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1907 - acc: 0.8117 - val_loss: 0.9589 - val_acc: 0.9351
Epoch 61/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1902 - acc: 0.8117 - val_loss: 0.9586 - val_acc: 0.9341
Epoch 62/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1897 - acc: 0.8115 - val_loss: 0.9584 - val_acc: 0.9341
Epoch 63/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1892 - acc: 0.8115 - val_loss: 0.9577 - val_acc: 0.9341
Epoch 64/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1887 - acc: 0.8115 - val_loss: 0.9582 - val_acc: 0.9351
Epoch 65/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1881 - acc: 0.8117 - val_loss: 0.9580 - val_acc: 0.9331
Epoch 66/400
3941/3941 [================

Epoch 118/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1613 - acc: 0.8148 - val_loss: 0.9584 - val_acc: 0.9351
Epoch 119/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1607 - acc: 0.8150 - val_loss: 0.9588 - val_acc: 0.9351
Epoch 120/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1602 - acc: 0.8153 - val_loss: 0.9591 - val_acc: 0.9351
Epoch 121/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1597 - acc: 0.8153 - val_loss: 0.9595 - val_acc: 0.9351
Epoch 122/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1591 - acc: 0.8168 - val_loss: 0.9594 - val_acc: 0.9351
Epoch 123/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1586 - acc: 0.8183 - val_loss: 0.9589 - val_acc: 0.9351
Epoch 124/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1581 - acc: 0.8181 - val_loss: 0.9592 - val_acc: 0.9351
Epoch 125/400
3941/3

3941/3941 [==============================] - 0s 14us/sample - loss: 1.1320 - acc: 0.8224 - val_loss: 0.9599 - val_acc: 0.9341
Epoch 177/400
3941/3941 [==============================] - 0s 13us/sample - loss: 1.1314 - acc: 0.8221 - val_loss: 0.9599 - val_acc: 0.9341
Epoch 178/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1310 - acc: 0.8221 - val_loss: 0.9604 - val_acc: 0.9341
Epoch 179/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1305 - acc: 0.8221 - val_loss: 0.9611 - val_acc: 0.9341
Epoch 180/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1301 - acc: 0.8224 - val_loss: 0.9605 - val_acc: 0.9320
Epoch 181/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1297 - acc: 0.8224 - val_loss: 0.9599 - val_acc: 0.9331
Epoch 182/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1291 - acc: 0.8224 - val_loss: 0.9617 - val_acc: 0.9320
Epoch 183/400
3941/3941 [=========

3941/3941 [==============================] - 0s 14us/sample - loss: 1.1024 - acc: 0.8277 - val_loss: 0.9643 - val_acc: 0.9300
Epoch 235/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1019 - acc: 0.8277 - val_loss: 0.9630 - val_acc: 0.9300
Epoch 236/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1014 - acc: 0.8277 - val_loss: 0.9633 - val_acc: 0.9300
Epoch 237/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1010 - acc: 0.8275 - val_loss: 0.9629 - val_acc: 0.9300
Epoch 238/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.1003 - acc: 0.8275 - val_loss: 0.9649 - val_acc: 0.9300
Epoch 239/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0998 - acc: 0.8277 - val_loss: 0.9651 - val_acc: 0.9300
Epoch 240/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0994 - acc: 0.8275 - val_loss: 0.9634 - val_acc: 0.9290
Epoch 241/400
3941/3941 [=========

3941/3941 [==============================] - 0s 14us/sample - loss: 1.0736 - acc: 0.8333 - val_loss: 0.9672 - val_acc: 0.9290
Epoch 293/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0731 - acc: 0.8333 - val_loss: 0.9672 - val_acc: 0.9290
Epoch 294/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0726 - acc: 0.8333 - val_loss: 0.9662 - val_acc: 0.9280
Epoch 295/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0723 - acc: 0.8325 - val_loss: 0.9667 - val_acc: 0.9300
Epoch 296/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0719 - acc: 0.8325 - val_loss: 0.9661 - val_acc: 0.9300
Epoch 297/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0714 - acc: 0.8328 - val_loss: 0.9659 - val_acc: 0.9280
Epoch 298/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0708 - acc: 0.8328 - val_loss: 0.9670 - val_acc: 0.9290
Epoch 299/400
3941/3941 [=========

3941/3941 [==============================] - 0s 14us/sample - loss: 1.0440 - acc: 0.8412 - val_loss: 0.9693 - val_acc: 0.9270
Epoch 351/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0434 - acc: 0.8422 - val_loss: 0.9718 - val_acc: 0.9270
Epoch 352/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0430 - acc: 0.8419 - val_loss: 0.9703 - val_acc: 0.9270
Epoch 353/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0424 - acc: 0.8417 - val_loss: 0.9697 - val_acc: 0.9270
Epoch 354/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0419 - acc: 0.8419 - val_loss: 0.9715 - val_acc: 0.9270
Epoch 355/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0414 - acc: 0.8419 - val_loss: 0.9706 - val_acc: 0.9270
Epoch 356/400
3941/3941 [==============================] - 0s 14us/sample - loss: 1.0408 - acc: 0.8422 - val_loss: 0.9713 - val_acc: 0.9270
Epoch 357/400
3941/3941 [=========

In [ ]:
html_clf.load('../saved_model/html_clf.h5')